In [7]:
# # Check the version of the installed Python interpreter
# !pip install requests beautifulsoup4
# !pip install selenium
# !pip install fake_useragent
# !pip install selenium-wire

In [8]:

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep

import requests
from bs4 import BeautifulSoup
from time import sleep
import re
import pandas as pd
import random
import copy

from package import generate_search_query, surf, reload_cookies, re_school_department, re_post_request


### 預設值

In [9]:
url = 'https://ndltd.ncl.edu.tw/' # = 碩博士論文網
cookie = "oMevOx" # = 預設 cookie

clear_excel_dict_template = {   
                                '計畫主持人': '',
                                '學校': '',
                                "碩士畢業學年度": "", 
                                "碩士畢業學校": "", 
                                "碩士指導教授":"",	
                                "碩士論文題目": "",
                                
                                "博士畢業學年度": "",
                                "博士畢業學校": "",
                                "博士指導教授": "",
                                "博士論文題目": ""
                            }

# - 要存擋的 Excel 資料
columns = list(clear_excel_dict_template.keys())
excel_df = pd.DataFrame(columns=columns)

# - 讀取上傳的 Excel 檔案
file_path = './NST.xlsx'
excel_data = pd.read_excel(file_path, sheet_name="研究人才")


### 爬蟲

In [10]:
for index, row in excel_data.iterrows():
    
    # @ 跳過
    if index >= 10: break
    
    student_name = row['計畫主持人']
    school_name = row['學校']
    
    # @ 準備 Excel 存擋資料
    temp_dict = clear_excel_dict_template.copy() # = 預設
    temp_dict['計畫主持人'] = student_name
    temp_dict['學校'] = school_name
    temp_dict["備註"] = ""
    
    # @ 準備爬蟲資料
    query = generate_search_query(student_name)
    
    retry = True
    error = 0
    
    while retry and error <= 2:
        
        # ~ 若成功跑完 try
        try:
            
            # - 爬蟲 post request
            cookie, rs, res_post, h1 = re_post_request(cookie, query, headers, h1) # ! re_post
            
            # - 找到共搜索幾筆
            soup = BeautifulSoup(res_post.text, 'html.parser')
            brwrestable = soup.find('table', {'class': 'brwrestable'})
            if brwrestable:
                brwreSpan = brwrestable.findAll("span", {"class": "etd_e"})
                search_counts = int([brwres.text.replace('\xa0', '') for brwres in brwreSpan if brwres.text != query][0])
                print(f"{student_name} 共有 {search_counts} 筆資料")
                
                temp_dict["查獲人數"] = f"{search_counts}"
            else:
                temp_dict["備註"] += "搜尋錯誤"
                break
                
            
            # - 找到內文
            PHD_count = 0
            temp_dict["查獲博士人數"] = f"{PHD_count}"
            if search_counts <= 10: 
                
                if search_counts > 2 :
                    temp_dict["備註"] += f"人數多於2人以上，跳過碩士學位"
                
                for r1 in range(1, int(search_counts) + 1):
                    
                    current_data_dict = {} # = 找到內文存成字典
                    
                    # - 爬蟲資料
                    res_get = surf(cookie, rs, r1, h1) # ! re_get
                    soup_content = BeautifulSoup(res_get.text, 'html.parser')

                    # - 內文找到格式
                    contentTable = soup_content.find('table', {'id': 'format0_disparea'})
                    if contentTable:
                        contents = [(content.find('th',{'class':'std1'}).text.replace(":", ""), content.find('td',{'class':'std2'}).text) 
                                    for content in contentTable.findAll('tr') 
                                    if not content.find("td", {"class": "push_td"}) and not content.find("img", {"alt": "被引用"})]

                        for column, value in contents:
                            current_data_dict[column] = value
                            
                            
                    # - 處理後綴
                    endWith = ""
                    if current_data_dict["學位類別"] == "博士":
                        PHD_count += 1
                        if PHD_count > 1:
                            endWith = f"_{PHD_count}"
                            
                        temp_dict["查獲博士人數"] = f"{PHD_count}"
                            
                    if (current_data_dict["學位類別"] == "碩士" and search_counts <= 2) or (current_data_dict["學位類別"] == "博士"):
                        # - 填入
                        temp_dict[f"{current_data_dict['學位類別']}畢業學年度" + endWith] = current_data_dict["畢業學年度"]
                        temp_dict[f"{current_data_dict['學位類別']}畢業學校" + endWith] = current_data_dict["校院名稱"] + "／" + current_data_dict["系所名稱"]
                        temp_dict[f"{current_data_dict['學位類別']}指導教授" + endWith] = current_data_dict["指導教授"]
                        if "論文名稱" in current_data_dict:
                            temp_dict[f"{current_data_dict['學位類別']}論文題目" + endWith] = current_data_dict["論文名稱"]
                        elif "論文名稱(外文)" in current_data_dict:
                            temp_dict[f"{current_data_dict['學位類別']}論文題目" + endWith] = current_data_dict["論文名稱(外文)"]
                            
            else:
                temp_dict["備註"] += f"人數過多，僅搜尋十筆內資料"
                    
                    
            # - 存擋
            temp_df = pd.DataFrame([temp_dict])
            excel_df = pd.concat([excel_df, temp_df], ignore_index=True)

            # - 結束
            rs.close()
            retry = False
            sleep(random.randint(2, 5))
            
        except Exception as e:
            
            print(f"{e} \n重新獲取 Cookies: {student_name},index: {index}", )
            cookie, rs, res_post, headers, h1 = reload_cookies(url, query) # ! reload cookies
            retry = True
            error += 1
            
    


excel_df
    

Brij Bhooshan Gupta 共有 0 筆資料
Hieu 共有 98 筆資料
Naveed Ishtiaq Chaudhary 共有 0 筆資料
丁川康 共有 1 筆資料
'學位類別' 
重新獲取 Cookies: 丁川康,index: 3
丁川康 共有 1 筆資料
丁信文 共有 1 筆資料
丁英智 共有 3 筆資料
丁國正 共有 1 筆資料
于天立 共有 0 筆資料
尤昌筧 共有 0 筆資料
尤信程 共有 0 筆資料
文武 共有 17 筆資料
方士豪 共有 7 筆資料
方文杰 共有 2 筆資料
方文杰 共有 2 筆資料
方文聘 共有 2 筆資料
方百立 共有 0 筆資料
方瓊瑤 共有 4 筆資料
水谷英二 共有 1 筆資料
王三元 共有 1 筆資料
王才沛 共有 0 筆資料
王才沛 共有 0 筆資料
王元凱 共有 1 筆資料
王文楓 共有 2 筆資料
王正豪 共有 2 筆資料
王正豪 共有 2 筆資料
王永聯 共有 2 筆資料
王任瓚 共有 2 筆資料
王任瓚 共有 2 筆資料
王孝熙 共有 1 筆資料
王希俊 共有 0 筆資料
王坤卿 共有 2 筆資料
王坤卿 共有 2 筆資料
王宗銘 共有 1 筆資料
王宗銘 共有 1 筆資料
王帝皓 共有 0 筆資料
王昭能 共有 2 筆資料
王昱舜 共有 3 筆資料
王昱鈞 共有 21 筆資料
王派洲 共有 0 筆資料
王禹超 共有 1 筆資料
王禹超 共有 1 筆資料
王科植 共有 1 筆資料
王英宏 共有 3 筆資料
王貞淑 共有 2 筆資料
王孫崇 共有 1 筆資料
王家祥 共有 17 筆資料
王家慶 共有 12 筆資料
王峻國 共有 3 筆資料
王能中 共有 2 筆資料
王國禎 共有 0 筆資料
王國禎 共有 0 筆資料
王淵弘 共有 0 筆資料
王祥安 共有 19 筆資料
王釧茹 共有 1 筆資料
王傑智 共有 1 筆資料
王堯天 共有 2 筆資料
王智昊 共有 1 筆資料
王隆仁 共有 1 筆資料
王順吉 共有 2 筆資料
王敬文 共有 13 筆資料
王敬文 共有 13 筆資料
王新民 共有 2 筆資料
王暉智 共有 2 筆資料
王毓淇 共有 2 筆資料
王煌城 共有 0 筆資料
王瑞堂 共有 4 筆資料
王瑞麟 共有 15 筆資料
王經篤 共有 2 筆資料
王聖文 共有 55 筆資料
王聖文 共有 

,計畫主持人,學校,碩士畢業學年度,碩士畢業學校,碩士指導教授,碩士論文題目,博士畢業學年度,博士畢業學校,博士指導教授,博士論文題目,...,查獲人數,查獲博士人數,博士畢業學年度_2,博士畢業學校_2,博士指導教授_2,博士論文題目_2,博士畢業學年度_3,博士畢業學校_3,博士指導教授_3,博士論文題目_3
0,Brij Bhooshan Gupta,亞洲大學資訊工程學系,,,,,,,,,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hieu,國立臺北科技大學電子工程系,,,,,,,,,...,98,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Naveed Ishtiaq Chaudhary,國立雲林科技大學前瞻學士學位學程,,,,,,,,,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,丁川康,國立清華大學動力機械工程學系,84,國立清華大學／動力機械學系,林士傑,以尺寸文字為基礎進行機械製圖尺寸線辨識之研究,,,,,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,丁信文,國立高雄科技大學電機與資訊學院電子工程系(建工校區),92,國立成功大學／電機工程學系碩博士班,劉濱達、張順志,應用於類比數位轉換器SNDR及ENOB測試之時域內建自我測試方法,,,,,...,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1256,饒建奇,淡江大學學校財團法人淡江大學電機工程學系,,,,,,,,,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1257,顧皓翔,國立臺灣海洋大學食品安全與風險管理研究所,,,,,,,,,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,欉振坤,國立勤益科技大學資訊工程系,,,,,,,,,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1259,龔于農,長庚大學新興病毒感染研究中心,,,,,,,,,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
excel_df.to_excel('NST_crawler.xlsx', index=False, engine='openpyxl', sheet_name="研究人才")

In [12]:

# soup = BeautifulSoup(res.text, 'html.parser')

# # 使用CSS選擇器找到所有具有特定class的<span>標籤
# spans = soup.select('td.std2 a.slink span.etd_d')

# for span in spans:
#     print(span.text)
